<a href="https://colab.research.google.com/github/ProJedi1234/Java-Process-Scheduler/blob/master/COVID19_News_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#COVID-19 News Extraction

##Install and setup News API

###Install News API

In [3]:
!pip install spacy
!pip install newsapi-python

In [4]:
import spacy
import newsapi
import pickle
import pandas as pd

###Download Language Model

In [5]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 8.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


###Import spaCy library

In [6]:
nlp_eng = spacy.load('en_core_web_sm')
newsapi = newsapi.NewsApiClient (api_key='3238bd5f6e6b4d08b9fd1b8ddf6b564d')

##Gather News

In [10]:
temp = newsapi.get_everything(q='coronavirus', language='en', from_param='2022-02-28', to='2022-03-27', sort_by='relevancy', page=1)

In [11]:
filename = 'articlesCOVID.pckl'
pickle.dump(temp, open(filename, 'wb'))
filename = 'articlesCOVID.pckl'
loaded_model = pickle.load(open(filename, 'rb'))
filepath = '/articlesCOVID.pckl'
pickle.dump(loaded_model, open(filepath, 'wb'))

In [12]:
print(temp)

{'status': 'ok', 'totalResults': 22239, 'articles': [{'source': {'id': None, 'name': 'New York Times'}, 'author': 'Lauren Hilgers, John Woo, Aaron Esposito, Jack D’Isidoro, Marion Lozano and Zachary Mouton', 'title': 'The Sunday Read: ‘Nurses Have Finally Learned What They’re Worth’', 'description': 'As the coronavirus spread, demand for nurses came from every corner. Some traveling jobs paid more than $10,000 a week. Will the boom last?', 'url': 'https://www.nytimes.com/2022/03/27/podcasts/the-daily/nursing-pandemic-sunday-read.html', 'urlToImage': 'https://static01.nyt.com/images/2022/02/06/magazine/27daily-audm-1/06mag-NSO-1-facebookJumbo.jpg', 'publishedAt': '2022-03-27T10:00:07Z', 'content': 'There are a lot of ways to listen to The Daily. Heres how.\r\nWe want to hear from you. Tune in, and tell us what you think. Email us at thedaily@nytimes.com. Follow Michael Barbaro on Twitter: @mikieb… [+501 chars]'}, {'source': {'id': None, 'name': 'New York Times'}, 'author': 'Joseph Berge

In [13]:
dados = []

for i, article in enumerate(temp['articles']):
  title = article['title']
  description = article['description']
  content = article['content']
  dados.append({'title':title, 'date':article["publishedAt"], 'desc':description, 'content':content})
df = pd.DataFrame(dados)
df = df.dropna()
df.head()

,title,date,desc,content
0,The Sunday Read: ‘Nurses Have Finally Learned ...,2022-03-27T10:00:07Z,"As the coronavirus spread, demand for nurses c...",There are a lot of ways to listen to The Daily...
1,"Rabbi Chaim Kanievsky, Venerated Talmudic Scho...",2022-03-18T20:26:55Z,"He initially insisted, “Canceling Torah study ...",Most significant was his ruling that instances...
2,Can Eric Adams Cheerlead New Yorkers Past the ...,2022-03-20T16:11:17Z,The mayor is singularly focused on bringing ba...,"Mr. Adams, a Democrat, has had regular convers..."
3,Moderna Asks F.D.A. to Authorize a Second Boos...,2022-03-18T09:08:37Z,The request is significantly broader than the ...,A man receiving a second booster of a coronavi...
4,China Reports Its First Covid Deaths in Over a...,2022-03-19T13:37:04Z,The two fatalities are the first coronavirus d...,Scarcely two months after the Omicron variant ...


In [20]:
def get_keywords_eng(text):
  result = []
  for token in text:
      if (token in nlp_eng.Defaults.stop_words or token in [',', '.',';']):
        continue
      if (token.pos_ in pos_tag):
        result.append(token.text)
      return result

In [39]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
words = []
for result in temp['articles']:
  words.extend(word_tokenize(result["content"]))

##Word Cloud

In [41]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [44]:
words = [w.lower() for w in words]
nltk.download('stopwords')
words = [w for w in words if w not in stopwords.words('english')]
words = [w for w in words if w.isalnum()]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [48]:
# import packages needed
from nltk import pos_tag
from nltk.corpus import wordnet 
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

# Create a function to derive given words POS
# Fcn source: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/)
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Now use NLTK's lemmatizer to update our list of words
words = [WordNetLemmatizer().lemmatize(w, get_wordnet_pos(w)) for w in words]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [50]:
# import wordcloud library
from wordcloud import WordCloud

# create a single string of space separated words
unique_string=(" ").join(words)

# create the word cloud and save to file
wordcloud = WordCloud(width = 1000, height = 500).generate(unique_string)
wordcloud.to_file("word_cloud.png")